In [1]:
%reset -fs

import pandas as pd
import numpy as np
from sklearn import metrics

from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
import matplotlib.pyplot as plt
import matplotlib

import seaborn as sns
%matplotlib inline
matplotlib.use('TkAgg')

# pd.set_option('max_columns', 100)
pd.set_option("display.max_colwidth", 1)

In [2]:
df = pd.read_csv('tweets_01-08-2021.csv', encoding='utf-8')

In [3]:
len(df)

56571

In [4]:
df.head(30)

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,Republicans and Democrats have both created our economic problems.,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,"I was thrilled to be back in the Great city of Charlotte, North Carolina with thousands of hardworking American Patriots who love our Country, cherish our values, respect our laws, and always put AMERICA FIRST! Thank you for a wonderful evening!! #KAG2020 https://t.co/dNJZfRsl9y",f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance court obtained by CBS News questions where there will be further disciplinary action and cho…,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,"The Unsolicited Mail In Ballot Scam is a major threat to our Democracy, &amp; the Democrats know it. Almost all recent elections using this system, even though much smaller &amp; with far fewer Ballots to count, have ended up being a disaster. Large numbers of missing Ballots &amp; Fraud!",f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,RT @MZHemingway: Very friendly telling of events here about Comey's apparent leaking to compliant media. If you read those articles and tho…,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
5,1217962723234983937,RT @WhiteHouse: President @realDonaldTrump announced historic steps to protect the Constitutional right to pray in public schools! https://…,t,f,Twitter for iPhone,0,25048,2020-01-17 00:11:56,f
6,1223640662689689602,Getting a little exercise this morning! https://t.co/fyAAcbhbgk,f,f,Twitter for iPhone,285863,30209,2020-02-01 16:14:02,f
7,1319501865625784320,https://t.co/4qwCKQOiOw,f,f,Twitter for iPhone,130822,19127,2020-10-23 04:52:14,f
8,1319500520126664705,https://t.co/VlEu8yyovv,f,f,Twitter for iPhone,153446,20275,2020-10-23 04:46:53,f
9,1319500501269041154,https://t.co/z5CRqHO8vg,f,f,Twitter for iPhone,102150,14815,2020-10-23 04:46:49,f


In [5]:
df.groupby('isFlagged').count()

,id,text,isRetweet,isDeleted,device,favorites,retweets,date
isFlagged,,,,,,,,
f,56267,56267,56267,56267,56267,56267,56267,56267
t,304,304,304,304,304,304,304,304


In [6]:
# Text preprocessing steps - remove numbers, URL, captial letters and punctuation
import re
import string

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub(r'[^\w\s]', ' ',str(x).lower())
no_url = lambda x: re.sub(r'http\S+', '', x)
clean = lambda x: x.replace('\n', '')
english_only = lambda string: re.sub(r'[^\x00-\x7f]', "", string)

df['text'] = df.text.map(alphanumeric).map(no_url).map(punc_lower).map(clean).map(english_only)
df.head(50)

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,republicans and democrats have both created our economic problems,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,i was thrilled to be back in the great city of charlotte north carolina with thousands of hardworking american patriots who love our country cherish our values respect our laws and always put america first thank you for a wonderful evening,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,rt cbs_herridge read letter to surveillance court obtained by cbs news questions where there will be further disciplinary action and cho,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,the unsolicited mail in ballot scam is a major threat to our democracy amp the democrats know it almost all recent elections using this system even though much smaller amp with far fewer ballots to count have ended up being a disaster large numbers of missing ballots amp fraud,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,rt mzhemingway very friendly telling of events here about comey s apparent leaking to compliant media if you read those articles and tho,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
5,1217962723234983937,rt whitehouse president realdonaldtrump announced historic steps to protect the constitutional right to pray in public schools,t,f,Twitter for iPhone,0,25048,2020-01-17 00:11:56,f
6,1223640662689689602,getting a little exercise this morning,f,f,Twitter for iPhone,285863,30209,2020-02-01 16:14:02,f
7,1319501865625784320,,f,f,Twitter for iPhone,130822,19127,2020-10-23 04:52:14,f
8,1319500520126664705,,f,f,Twitter for iPhone,153446,20275,2020-10-23 04:46:53,f
9,1319500501269041154,,f,f,Twitter for iPhone,102150,14815,2020-10-23 04:46:49,f


In [7]:
df['text'] = df['text'].apply(lambda x: x.strip()).replace('', np.nan) # fill empty cells w/ Null values

In [8]:
df.head(50)

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,republicans and democrats have both created our economic problems,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,i was thrilled to be back in the great city of charlotte north carolina with thousands of hardworking american patriots who love our country cherish our values respect our laws and always put america first thank you for a wonderful evening,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,rt cbs_herridge read letter to surveillance court obtained by cbs news questions where there will be further disciplinary action and cho,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,the unsolicited mail in ballot scam is a major threat to our democracy amp the democrats know it almost all recent elections using this system even though much smaller amp with far fewer ballots to count have ended up being a disaster large numbers of missing ballots amp fraud,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,rt mzhemingway very friendly telling of events here about comey s apparent leaking to compliant media if you read those articles and tho,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
5,1217962723234983937,rt whitehouse president realdonaldtrump announced historic steps to protect the constitutional right to pray in public schools,t,f,Twitter for iPhone,0,25048,2020-01-17 00:11:56,f
6,1223640662689689602,getting a little exercise this morning,f,f,Twitter for iPhone,285863,30209,2020-02-01 16:14:02,f
7,1319501865625784320,NaN,f,f,Twitter for iPhone,130822,19127,2020-10-23 04:52:14,f
8,1319500520126664705,NaN,f,f,Twitter for iPhone,153446,20275,2020-10-23 04:46:53,f
9,1319500501269041154,NaN,f,f,Twitter for iPhone,102150,14815,2020-10-23 04:46:49,f


In [9]:
df = df.dropna() #drop null rows
df.head(50)

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,republicans and democrats have both created our economic problems,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,i was thrilled to be back in the great city of charlotte north carolina with thousands of hardworking american patriots who love our country cherish our values respect our laws and always put america first thank you for a wonderful evening,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,rt cbs_herridge read letter to surveillance court obtained by cbs news questions where there will be further disciplinary action and cho,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,the unsolicited mail in ballot scam is a major threat to our democracy amp the democrats know it almost all recent elections using this system even though much smaller amp with far fewer ballots to count have ended up being a disaster large numbers of missing ballots amp fraud,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,rt mzhemingway very friendly telling of events here about comey s apparent leaking to compliant media if you read those articles and tho,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
5,1217962723234983937,rt whitehouse president realdonaldtrump announced historic steps to protect the constitutional right to pray in public schools,t,f,Twitter for iPhone,0,25048,2020-01-17 00:11:56,f
6,1223640662689689602,getting a little exercise this morning,f,f,Twitter for iPhone,285863,30209,2020-02-01 16:14:02,f
11,1319496349092511744,rt erictrump,t,f,Twitter for iPhone,0,8921,2020-10-23 04:30:19,f
12,1236502342121541632,rt gopchairwoman the economic boom continues jobs added in february jobs added since realdonaldtrump was electedunemploym,t,f,Twitter for iPhone,0,8681,2020-03-08 04:01:46,f
13,1225835449379258368,rt tomfitton vindman s behavior is a scandal he should be removed from the realdonaldtrump white house asap to protect our foreign poli,t,f,Twitter for iPhone,0,7679,2020-02-07 17:35:20,f


In [10]:
from datetime import datetime as dt

In [11]:
df["date"] = pd.to_datetime(df.date , format="%Y/%m/%d %H:%M:%S") #create date_time column

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets

from corextopic import corextopic as ct
from corextopic import vis_topic as vt

In [55]:
vectorizer = CountVectorizer(max_features=20000,
                             stop_words='english', 
                             token_pattern="\\b[a-z][a-z]+\\b",
                             binary=True, max_df=.8)

doc_word = vectorizer.fit_transform(df.text)
words = list(np.asarray(vectorizer.get_feature_names_out()))

In [56]:
doc_word.shape

(55247, 20000)

In [62]:
topic_model = ct.Corex(n_hidden=10, words=words, seed=1)
topic_model.fit(doc_word, words=words, docs=df.text)

In [63]:
# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: fake,news,media,fbi,impeachment,witch,hunt,schiff,collusion,mueller
1: united,states,border,military,wall,endorsement,law,strong,federal,vets
2: realdonaldtrump,democrats,biden,china,amp,joe,thanks,party,republican,dems
3: hillary,obama,thank,bad,clinton,crooked,said,just,didn,obamacare
4: rt,whitehouse,great,teamtrump,president,gopchairwoman,dbongino,potus,danscavino,breaking
5: golf,course,hotel,trump,scotland,doral,tower,trumpdoral,chicago,macys
6: carolina,today,join,crowd,rally,north,live,south,tomorrow,honor
7: new,york,jobs,economy,market,record,unemployment,deal,stock,economic
8: apprentice,tonight,apprenticenbc,interview,celebapprentice,celebrity,foxnews,interviewed,nbc,season
9: god,vote,bless,paul,hall,virginia,swamp,maga,wisconsin,jon


In [64]:
# Let's check out topic : graphics
topic_model.get_top_docs(topic=5, n_docs=2)

[('golfswingshirt   realdonaldtrump looking forward to playing trump national la on tuesday with padraig harrington      say hi to padraig',
  0.0),
 ('steps away from waikiki s famous beaches   trumpwaikiki is hawaii s top destination  w our signature amenities',
  0.0)]

In [66]:
predictions = pd.DataFrame(topic_model.predict(doc_word), columns=['topic'+str(i) for i in range(10)])
predictions.head(3)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,False,False,False,False,False,False,False,True,False,False
1,False,True,True,False,False,False,True,False,False,False
2,True,False,True,False,True,False,False,False,False,False


In [67]:
print(topic_model.tcs)
print(topic_model.tc)

[0.54674561 0.51504964 0.37071296 0.31824684 0.31630055 0.29168116
 0.23395932 0.22482351 0.21033574 0.10229058]
3.130145903406108


In [76]:
vectorizer2 = TfidfVectorizer(max_features=20000,
                             stop_words='english',max_df=0.8)

doc_word2 = vectorizer.fit_transform(df.text)

print(doc_word2.shape)

words2 = list(np.asarray(vectorizer.get_feature_names()))

(55247, 20000)


In [77]:
topic_model2 = ct.Corex(n_hidden=10, words=words2, seed=1)
topic_model2.fit(doc_word2, words=words2, docs=df.text)

In [79]:
topics2 = topic_model2.get_topics()
for n,topic in enumerate(topics2):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: thanks,realdonaldtrump,thank,apprenticenbc,celebapprentice,apprentice,makeamericagreatagain,tonight,enjoy,democrats
1: golf,course,hotel,links,luxury,club,trumpdoral,international,scotland,trumpchicago
2: fbi,comey,dossier,mueller,mccabe,steele,collusion,tomfitton,doj,fisa
3: prime,minister,abeshinzo,japan,abe,nhc_atlantic,spacex,nasa,dorian,tropical
4: dphilbs,pearl_brendan,ufcwashington,ufcdc,thugnastymma,realdonaldtrumpwait,payrolls,adp,mitchell,batteries
5: pablo,picasso,eugene,dunn,pam,bondi,foundational,prayfornashville,lasvegassun,katrina
6: huffpostmedia,geoscarborough,teapartyexpress,teapartyorg,moyers,jeremiahwright,imagined,confidently,rjispy,billmoyers
7: tzu,defence,invincibility,waits,hustles,sun,mastering,rdy,realchrisrufo,realclearpolitics
8: aboard,ranting,rapi,rapidcraft,rashidatlaib,rasmussennews,ratted,raxsondfs,raytheon,rcp
9: joeho,kathviner,chutzpah,wusses,weasels,regional,reidepstein,relentlessly,relevance,reminding


In [80]:
topic_model2.get_top_docs(topic=5, n_docs=2)

[('rt  alwaysactions  the reason why presidentdonald j  trump was electedas president  and the reasonwhy he will be elected again',
  -5.213707818599311e-07),
 ('i agree katrina  pam bondi is a great womem', -0.00018064478549217888)]

In [81]:
print(topic_model2.tcs)
print(topic_model2.tc)

[8.64739284e-02 2.18678226e-02 1.50677089e-02 7.87052307e-04
 3.56568738e-05 2.11182289e-05 1.62769422e-05 9.02564298e-06
 1.75406118e-06 6.22988065e-07]
0.12428096700004425


In [ ]:
# df['text'] =  df['text'].apply(lambda sentence: [lemmatizer.lemmatize for w in word_tokenize(corpus)])

In [ ]:
# stop_words = ENGLISH_STOP_WORDS

In [68]:
# corpus = df['text']

In [ ]:
tweet_topic_matrix_df = pd.DataFrame(tweet_topic_matrix).add_prefix('topic_')
tweet_topic_matrix_df['tweets'] = df['text']
tweet_topic_matrix_df.head()

In [ ]:
word_topic_matrix_df = pd.DataFrame(nmf.components_, columns=vocab).T.add_prefix('topic_')
# word_topic_matrix_df.tail(5)

In [ ]:
# for tweet in tweet_topic_matrix_df.sort_values(by='topic_0', ascending=False).head(10)['tweets'].values:
#     print(tweet)
#     print()

In [ ]:
word_topic_matrix_df.sort_values(by='topic_0', ascending=False).head(10)

In [ ]:
def top_tweets(tweet_topic_matrix_df, topic, n_tweets):
    return (tweet_topic_matrix_df
            .sort_values(by=topic, ascending=False)
            .head(n_tweets)['tweets']
            .values)

def top_words(word_topic_matrix_df, topic, n_words):
    return (word_topic_matrix_df
            .sort_values(by=topic, ascending=False)
            .head(n_words))[topic]

In [ ]:
# for tweet in top_tweets(tweet_topic_matrix_df, 'topic_1', 15):
#     print(tweet)
#     print()

In [ ]:
top_words(word_topic_matrix_df, 'topic_0', 10)

In [ ]:
top_words(word_topic_matrix_df, 'topic_1', 10)

In [ ]:
top_words(word_topic_matrix_df, 'topic_2', 10)

In [ ]:
top_words(word_topic_matrix_df, 'topic_3', 10)

In [ ]:
# for tweet in top_tweets(tweet_topic_matrix_df, 'topic_2', 5):
#     print(tweet)

In [ ]:
# for tweet in top_tweets(tweet_topic_matrix_df, 'topic_2', 5):
#     print(tweet)
#     print()

In [83]:
df_before = df[(df['date'] <= "2017-01-19")]
df_before

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,republicans and democrats have both created our economic problems,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1601,336155055575150594,yes,f,f,Twitter for Android,0,1,2013-05-19 16:23:05,f
1615,274563166384582657,thanks you will love it,f,f,Twitter Web Client,1,4,2012-11-30 17:18:54,f
1620,274562988646727681,celebapprentice bretmichaels you won t be disappointed,f,f,Twitter Web Client,0,3,2012-11-30 17:18:12,f
1696,284759862976581632,wandasalley thanks have fun,f,f,Twitter Web Client,1,2,2012-12-28 20:36:56,f
...,...,...,...,...,...,...,...,...,...
45874,815449868739211265,rt donaldjtrumpjr happy new year everyone newyear family vacation familytime,t,f,Twitter for iPhone,0,5548,2017-01-01 06:49:33,f
45875,815433444591304704,rt erictrump was such an incredible year for our entire family my beautiful wife laraleatrump made it even better,t,f,Twitter for iPhone,0,5601,2017-01-01 05:44:17,f
45876,815433217595547648,rt reince happy new year god s blessings to you all looking forward to incredible things in realdonaldtrump will make america,t,f,Twitter for iPhone,0,5811,2017-01-01 05:43:23,f
45877,815432169464197121,rt danscavino on behalf of our next potus amp teamtrump happynewyear america t co,t,f,Twitter for iPhone,0,4562,2017-01-01 05:39:13,f


In [84]:
df_after = df[(df['date'] > "2017-01-20")]
df_after

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
1,1234653427789070336,i was thrilled to be back in the great city of charlotte north carolina with thousands of hardworking american patriots who love our country cherish our values respect our laws and always put america first thank you for a wonderful evening,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,rt cbs_herridge read letter to surveillance court obtained by cbs news questions where there will be further disciplinary action and cho,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,the unsolicited mail in ballot scam is a major threat to our democracy amp the democrats know it almost all recent elections using this system even though much smaller amp with far fewer ballots to count have ended up being a disaster large numbers of missing ballots amp fraud,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,rt mzhemingway very friendly telling of events here about comey s apparent leaking to compliant media if you read those articles and tho,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
5,1217962723234983937,rt whitehouse president realdonaldtrump announced historic steps to protect the constitutional right to pray in public schools,t,f,Twitter for iPhone,0,25048,2020-01-17 00:11:56,f
...,...,...,...,...,...,...,...,...,...
56566,1319485303363571714,rt randpaul i don t know why joebiden thinks he can continue to lie about this he wants to ban fracking and end all fossil fuels like,t,f,Twitter for iPhone,0,20683,2020-10-23 03:46:25,f
56567,1319484210101379072,rt elisestefanik president realdonaldtrump excels at communicating directly to the american people joe biden communicates to the dc b,t,f,Twitter for iPhone,0,9869,2020-10-23 03:42:05,f
56568,1319444420861829121,rt teamtrump live presidential debate text vote to,t,f,Twitter for iPhone,0,8197,2020-10-23 01:03:58,f
56569,1319384118849949702,just signed an order to support the workers of delphi corporation and make sure that we protect the pensions of all american workers obama biden failed american workers and failed the workers of delphi i always put american workers first,f,f,Twitter for iPhone,176289,36001,2020-10-22 21:04:21,f


In [87]:
df_before2 = df_before['text']
# df_before2.iloc[0]
df_after2 = df_after['text']

In [88]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [89]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# for i in 

In [98]:
for text in df_before2, df_after2:
    score_before = analyzer.polarity_scores(df_before2.iloc[0])
    score_after = analyzer.polarity_scores(df_after2.iloc[0])

    print(score_before)
    print(score_after)

{'neg': 0.231, 'neu': 0.598, 'pos': 0.171, 'compound': -0.1779}
{'neg': 0.0, 'neu': 0.561, 'pos': 0.439, 'compound': 0.9771}
{'neg': 0.231, 'neu': 0.598, 'pos': 0.171, 'compound': -0.1779}
{'neg': 0.0, 'neu': 0.561, 'pos': 0.439, 'compound': 0.9771}


In [93]:
score_after = analyzer.polarity_scores(df_after2.iloc[0])
print(score_after)

{'neg': 0.0, 'neu': 0.561, 'pos': 0.439, 'compound': 0.9771}


In [115]:
k1 = df[(df['text'] == 'didn')]
k1

AttributeError: 'Series' object has no attribute 'find'

In [117]:
# looking at word in text
k1 = df[(df['text'].str.contains('didn'))]
k1

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
64,1317061707064279047,rt randpaul somebody is mad that realdonaldtrump didn t start any wars and wants to leave the ones we are in vote biden harris if you w,t,f,Twitter for iPhone,0,17819,2020-10-16 11:15:55,f
79,1319495252672012289,rt nra president realdonaldtrump won the debate why didn t you do that four years ago joebiden i ran because of you because you,t,f,Twitter for iPhone,0,8320,2020-10-23 04:25:57,f
98,1325961445062938625,as i have long said pfizer and the others would only announce a vaccine after the election because they didn t have the courage to do it before likewise the us_fda should have announced it earlier not for political purposes but for saving lives,f,f,Twitter for iPhone,292245,59870,2020-11-10 00:40:18,f
104,1325962203346972678,the us_fda and the democrats didn t want to have me get a vaccine win prior to the election so instead it came out five days later as i ve said all along,f,f,Twitter for iPhone,378553,71310,2020-11-10 00:43:19,f
108,1328330541494624257,rt rudygiuliani the only thing that would have surprised me is if philly didn t cheat on a big scale in the election it would have,t,f,Twitter for iPhone,0,21602,2020-11-16 13:34:14,f
...,...,...,...,...,...,...,...,...,...
56167,1092807525102247938,i see schumer is already criticizing my state of the union speech even though he hasn t seen it yet he s just upset that he didn t win the senate after spending a fortune like he thought he would too bad we weren t given more credit for the senate win by the media,f,f,Twitter for iPhone,131598,24648,2019-02-05 15:29:52,f
56235,1089339681941147648,about all of the one sided fake media coverage collusion with crooked h that i had to endure during my very successful presidential campaign what about the now revealed bias by facebook and many others roger stone didn t even work for me anywhere near the election,f,f,Twitter for iPhone,91105,19814,2019-01-27 01:49:53,f
56408,1084786495398072320,rt gopchairwoman it didn t get the attention it deserved but realdonaldtrump recently signed a bill into law that will empower women al,t,f,Twitter for iPhone,0,9789,2019-01-14 12:17:09,f
56436,1084062079907229697,funny thing about james comey everybody wanted him fired republican and democrat alike after the rigged amp botched crooked hillary investigation where she was interviewed on july weekend not recorded or sworn in and where she said she didn t know anything a lie,f,f,Twitter for iPhone,77448,17571,2019-01-12 12:18:35,f
